You can access the accompanying video here: 

<a href="https://blinks.bloomberg.com/screens/PLYR%20VOD%20361667896"> Bond Spread Z-Scores and Heatmaps (19 min)</a>

In [1]:
import bql 
import pandas as pd
import bqplot as bqp

bq = bql.Service()

In [2]:
members = bq.univ.members('luaitruu index')

dates = bq.func.range(start='2019-12-31',end='2020-04-24')

spread = bq.func.dropna(bq.data.spread(dates=dates))
net_chg = bq.func.net_chg(spread)

z_score = bq.func.round(bq.func.last(bq.func.z_score(spread)),2)

sector = bq.data.classification_name('BCLASS','3')
rating = bq.data.bb_composite()

net_chg_grouping = bq.func.group(net_chg,[rating,sector])
median_chg = bq.func.median(net_chg_grouping)

z_score_grouping = bq.func.group(z_score,[rating,sector])
median_z_score = bq.func.median(z_score_grouping)

fields = {'Median Spread Change': median_chg, 'Median Z Score': median_z_score}

request = bql.Request(members,fields)
response = bq.execute(request)

In [3]:
dataframe = pd.concat([x.df()[x.name] for x in response],axis=1)

dataframe.head()

,Median Spread Change,Median Z Score
ID,,
A+:Capital Goods,52.862167,0.070
A+:Consumer Cyclical,69.372589,0.250
A+:Consumer Non-Cyclical,47.896709,0.125
A+:Energy,101.920727,0.280
A+:Other Industrial,103.787331,0.750


In [4]:
dataframe['Rating'] = [x.split(':')[0] for x in dataframe.index]
dataframe['Sector'] = [x.split(':')[1] for x in dataframe.index]

dataframe.head()

,Median Spread Change,Median Z Score,Rating,Sector
ID,,,,
A+:Capital Goods,52.862167,0.070,A+,Capital Goods
A+:Consumer Cyclical,69.372589,0.250,A+,Consumer Cyclical
A+:Consumer Non-Cyclical,47.896709,0.125,A+,Consumer Non-Cyclical
A+:Energy,101.920727,0.280,A+,Energy
A+:Other Industrial,103.787331,0.750,A+,Other Industrial


In [5]:
import ipywidgets as ipw
import bqplot as bqp
import pandas as pd
import numpy as np

# Columns that will appear in the dropdown widgets
X_COLUMNS = ['Sector']
Y_COLUMNS = ['Rating']
VALUE_COLUMNS = ['Median Spread Change','Median Z Score']

# We need to reshape the data to create our plot
processed_dataframe = dataframe.pivot(index='Rating',
                                      columns='Sector',
                                      values='Median Spread Change')

label_df = pd.melt(processed_dataframe.reset_index(),
                   id_vars=[Y_COLUMNS[0]],
                   value_name=VALUE_COLUMNS[0])

# Create scales
scale_color = bqp.ColorScale(colors=['#FF1E3E', '#1a1a1a', '#30C030'])
scale_x = bqp.OrdinalScale()
scale_y = bqp.OrdinalScale()

# Create axes
axis_x = bqp.Axis(orientation='horizontal',
                  scale=scale_x,
                  label=X_COLUMNS[0],
                  grid_lines='none',
                  label_offset='3em')

axis_y = bqp.Axis(orientation='vertical',
                  scale=scale_y,
                  label=Y_COLUMNS[0],
                  grid_lines='none',
                  label_offset='6em')

# Create mark
grid_map = bqp.GridHeatMap(color=processed_dataframe,
                           row=processed_dataframe.index.astype(str),
                           column=processed_dataframe.columns,
                           scales={'color': scale_color,
                                   'row': scale_y,
                                   'column': scale_x},
                           font_style={'font-size': '12px',
                                       'fill':'black',
                                       'font-weight': 'bold'},
                           interactions={'click': 'select'},
                           selected_style={'stroke': '#ffffff',
                                           'stroke-width': 2},
                           opacity=1,
                           stroke='#626262')

mark_label = bqp.Label(x=label_df[X_COLUMNS[0]],
                       y=label_df[Y_COLUMNS[0]],
                       text=['{0:.3f}'.format(val)
                            for val in label_df[VALUE_COLUMNS[0]]],
                       scales={'x': scale_x, 'y': scale_y},
                       colors=['white'],
                       default_size=14,
                       font_weight='normal',
                       align='middle')

# Create figure
figure = bqp.Figure(marks=[grid_map, mark_label],
                    axes=[axis_x, axis_y],
                    title='Fixed Income Heat Map:',
                    title_style={'font-size': '22px'},
                    layout={'width':'auto', 'height': '500px'},
                    fig_margin={'top': 50, 'bottom': 75,
                                'left': 120, 'right': 50},
                    padding_y=0.0,
                    padding_x=0.0)

# HTML widget to display selection
html = ipw.HTML()

# Dropdown widgets
dropdown_x = ipw.Dropdown(description='X axis',
                          options=X_COLUMNS,
                          value=X_COLUMNS[0])
dropdown_y = ipw.Dropdown(description='Y axis',
                          options=Y_COLUMNS,
                          value=Y_COLUMNS[0])
dropdown_value = ipw.Dropdown(description='Value',
                              options=VALUE_COLUMNS,
                              value=VALUE_COLUMNS[0])

# Callback functions
def handle_selection(self, evt):
    html.value = 'Clicked on: {}'.format(evt['data'])

def update_plot(evt=None):
    if evt['new'] is not None:
        processed_dataframe = dataframe.pivot(index=dropdown_y.value,
                                              columns=dropdown_x.value,
                                              values=dropdown_value.value)

        label_df = pd.melt(processed_dataframe.reset_index(),
                   id_vars=[dropdown_y.value],
                   value_name=dropdown_value.value)

        grid_map.color = processed_dataframe
        grid_map.column = processed_dataframe.columns
        grid_map.row = processed_dataframe.index
        axis_x.label = dropdown_x.value
        axis_y.label = dropdown_y.value
        mark_label.x = label_df[dropdown_x.value]
        mark_label.y = label_df[dropdown_y.value]
        mark_label.text = ['{0:.3f}'.format(val)
                          for val in label_df[dropdown_value.value]]

# Event handlers for heatmap clicks and dropdown updates
grid_map.on_element_click(handle_selection)
dropdown_x.observe(update_plot, names=['value'])
dropdown_y.observe(update_plot, names=['value'])
dropdown_value.observe(update_plot, names=['value'])

# Display the figure
ipw.VBox([figure,
          ipw.HBox([dropdown_x, dropdown_y, dropdown_value]),
          html])
